In [63]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import root

## Non-closed-form solution
The non-closed-form solution for probability of default (PD) in the Merton model is given by:

$$ E = V \Phi(d_1) - D e^{-rT}\Phi(d_2) $$
$$ \sigma_E = \frac{V}{E} \Phi(d_1)\sigma_V $$

where:
- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution
- $ d_1 = \frac{\ln\frac{V}{D} + (r + 0.5{\sigma_V}^2)T}{\sigma_V\sqrt{T}} $
- $ d_2 = d_1 - \sigma_V\sqrt{T} $
- $ E $ is the equity value
- $ \sigma_E $ is the equity volatility 
- $ V $ is the asset value. This is unknown and solved for.
- $ \sigma_V $ is the asset volatility. This is unknown and solved for.
- $ D $ is the debt face value.
- $ r $ is the risk-free rate.
- $ T $ is the time to maturity.

These two functions are used to solve for $ V $ and $ \sigma_V $. Once that is done, the following equation is used to find probability of default:

$$ PD = \Phi\left( \frac{\ln\left(\frac{D}{V}\right) - \left( r - \frac{1}{2} \sigma_V^2 \right) T}{\sigma_V \sqrt{T}} \right)$$



where:

- $\Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution  
- $ V $ is the value of the firm's assets  
- $ D $ is the face value of debt 
- $ r $ is the risk-free interest rate  
- $ \sigma_V $ is the volatility of the firm's assets  
- $ T $ is the time to maturity




In [64]:
def compute_merton_pd(E, sigma_E, D, r, T=1.0, verbose=False):    

    def equations(vars):
        V, sigma_V = vars
        if V <= 0 or sigma_V <= 0:
            return 1e10, 1e10  # large penalty for invalid values
        d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
        d2 = d1 - sigma_V * np.sqrt(T)
        eq1 = V * norm.cdf(d1) - D * np.exp(-r * T) * norm.cdf(d2) - E
        eq2 = (V / E) * norm.cdf(d1) * sigma_V - sigma_E
        return [eq1, eq2]

    V0 = E + D
    sigma_V0 = sigma_E
    result = root(equations, [V0, sigma_V0], method='hybr')

    if result.success:
        V_opt, sigma_V_opt = result.x
        d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))
        pd = norm.cdf(-d2)
        return pd
    else:
        if verbose:
            print("Failure for firm:", E, sigma_E, D, r)
            print("Message:", result.message)
        return np.nan

In [ ]:
df= pd.read_csv('../data/clean_data.csv')
df['merton_pd'] = df.apply(
    lambda row: compute_merton_pd(
        E=row['market_cap'],
        sigma_E=row['equity_volatility'],
        D=row['total_debt'],
        r=row['rf'] / 100,  # Convert percentage to decimal
        verbose=True
    ),
    axis=1
)
print(df)

/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_95820/2908855909.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(V / D) + (r + 0.5 * sigma_V**2) * T) / (sigma_V * np.sqrt(T))
/var/folders/jx/vhdxb7ks06710zlsb29bkz000000gn/T/ipykernel_95820/2908855909.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  d2 = (np.log(V_opt / D) + (r - 0.5 * sigma_V_opt ** 2) * T) / (sigma_V_opt * np.sqrt(T))


Failure for firm: 34143950000.0 0.1486003368877531 256050000.0 0.0273
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 12798849840.0 1.184655266718994 133929500000.0 0.0347
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 10083127500.0 1.2063649383315511 133929500000.0 0.0348
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 5512109699.999999 1.3433913552168046 133929500000.0 0.0341
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 7232963460.0 1.3732017429642132 133929500000.0 0.0354
Message: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
Failure for firm: 10352010900.0 1.4225119557038712 1339295000

In [85]:
df['pd_valid'] = df['merton_pd'].notna()
df.to_csv("merton_model_output.csv", index=False)

In [86]:
df['pd_valid'].value_counts()

pd_valid
True     438957
False      2093
Name: count, dtype: int64

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print(df[df['merton_pd'].isna() == True])
(df[df['merton_pd'].isna() == True]).to_csv("nans.csv", index=False)

pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

              date  permno   tic                          conm        PRC  \
20449   2013-07-05   44644   ADP     AUTOMATIC DATA PROCESSING   70.75000   
23773   2008-09-15   66800   AIG  AMERICAN INTERNATIONAL GROUP    4.76000   
23774   2008-09-16   66800   AIG  AMERICAN INTERNATIONAL GROUP    3.75000   
23775   2008-09-17   66800   AIG  AMERICAN INTERNATIONAL GROUP    2.05000   
23776   2008-09-18   66800   AIG  AMERICAN INTERNATIONAL GROUP    2.69000   
23777   2008-09-19   66800   AIG  AMERICAN INTERNATIONAL GROUP    3.85000   
23778   2008-09-22   66800   AIG  AMERICAN INTERNATIONAL GROUP    4.72000   
23779   2008-09-23   66800   AIG  AMERICAN INTERNATIONAL GROUP    5.00000   
23780   2008-09-24   66800   AIG  AMERICAN INTERNATIONAL GROUP    3.31000   
23781   2008-09-25   66800   AIG  AMERICAN INTERNATIONAL GROUP    3.02000   
23782   2008-09-26   66800   AIG  AMERICAN INTERNATIONAL GROUP    3.15000   
23783   2008-09-29   66800   AIG  AMERICAN INTERNATIONAL GROUP    2.50000   

In [83]:
nans= pd.read_csv('nans.csv')
print(nans['leverage'].describe())
print()
print(nans['equity_volatility'].describe())

count    2093.000000
mean        0.316575
std         0.144230
min         0.007935
25%         0.203856
50%         0.286170
75%         0.459226
max         0.575617
Name: leverage, dtype: float64

count    2093.000000
mean        1.240458
std         0.902950
min         0.129284
25%         0.327642
50%         1.273436
75%         1.604754
max         3.528001
Name: equity_volatility, dtype: float64


In [84]:
print(df['leverage'].describe())
print()
print(df['equity_volatility'].describe())

count    441050.000000
mean          0.156678
std           0.099050
min           0.000000
25%           0.089657
50%           0.142806
75%           0.206324
max           0.575617
Name: leverage, dtype: float64

count    441050.000000
mean          2.226904
std          14.809782
min           0.090416
25%           0.189931
50%           0.243494
75%           0.330184
max         116.571964
Name: equity_volatility, dtype: float64
